In [1]:
# Standard libraries
import datetime as dt
import os

# Third-party libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt

# Local libraries
from utilities import netcdf


In [2]:
# Constants
ROOT_DIR = os.path.join(os.getcwd(), "data")
HAD_DIR = os.path.join(ROOT_DIR, "intermediate/hadukgrid")
GEO_DIR = os.path.join(ROOT_DIR, "processed/space")
VAR_DIR = os.path.join(ROOT_DIR, "processed/variable")
OUT_DIR = os.path.join(ROOT_DIR, "processed/climate")


In [3]:
# Climate data
NC_FILES = netcdf.list_files(HAD_DIR, path=True)
NC_FILES[0]


'/home/mikeblackett/Documents/Dev/climate-watch-uk-server/python/data/intermediate/hadukgrid/groundfrost-hadgriduk-1km.nc'

In [4]:
# Spatial data
uk = gpd.read_file(os.path.join(GEO_DIR, "uk.geojson"))
countries = gpd.read_file(os.path.join(GEO_DIR, "country.geojson"))
regions = gpd.read_file(os.path.join(GEO_DIR, "region.geojson"))

space = pd.concat([uk, countries, regions]).drop_duplicates("id")


In [5]:
def create_mask(gdf, ds, data_var):
    """Returns a data array with sizes equal to `ds`"""
    mask_da = xr.ones_like(ds[data_var].isel({"time": 0}))
    mask_da.name = "mask"
    mask_da = mask_da.rio.write_crs(ds.rio.crs)

    return xr.concat(
        [
            (mask_da.rio.clip([item.geometry], drop=False) == 1).expand_dims(
                dim={"space": [item.id]}
            )
            for item in gdf.itertuples()
        ],
        dim="space",
    )


In [6]:
def calc_spatial_mean(ds, mask, dims):
    return ds.where(mask).mean(dims)


In [7]:
def to_table(ds, data_var):
    df = (
        ds[data_var]
        .to_dataframe()[data_var]
        .reset_index()
        .rename(
            columns={data_var: "value", "time": "timeId", "space": "spaceId"}
        )
    )
    df["timeId"] = df.apply(
        lambda row: int(row.timeId.date().strftime("%Y%m")), axis=1
    )
    df.insert(loc=0, column="variableId", value=[data_var] * len(df))
    return df


In [8]:
Y_COORD = "projection_y_coordinate"
X_COORD = "projection_x_coordinate"

for fp in NC_FILES:
    with xr.open_dataset(fp, decode_coords="all", chunks="auto") as ds:
        DATA_VAR = list(ds.data_vars)[0]
        mask = create_mask(space, ds, DATA_VAR)
        mean = calc_spatial_mean(ds, mask, [X_COORD, Y_COORD])
        df = to_table(mean, DATA_VAR)
        df.to_csv(os.path.join(OUT_DIR, DATA_VAR + ".csv"), index=True)
